# Dataset Cleanup, Profiling and EDA
This notebook provides the data cleaning, profiling and exploratory data analysis (EDA) methods used in the project.  Those methods include the following steps:

1. Initial dataset import
2. Data cleanup
3. Profiling 
4. EDA

In [10]:
import pandas as pd

In [11]:
url = 'https://raw.githubusercontent.com/joelmsherman/ChatGPT-Topic-Modeling/master/Data/tweets.csv'
df = pd.read_csv(url)

### 1. Cleanup

In [12]:
# Delete unnecessary columns
df.drop(['hashtags', 'source', 'user_description', 'user_location', 'user_favourites'], axis=1, inplace=True)

# Queue up datetime types
df['date'] = pd.to_datetime(df['date'], infer_datetime_format=True, errors='coerce')
df['user_created'] = pd.to_datetime(df['user_created'], infer_datetime_format=True, errors='coerce')

# Change dtype of followers and friends to int; set Nan to zero
df['user_followers'] = df['user_followers'].fillna(0).astype('int')
df['user_friends'] = df['user_friends'].fillna(0).astype('int')

# Add a user duration calculated column and convert the timedelta to an int
df.insert(5, 'user_duration_days', 0)
df['user_duration_days'] = df['date'] - df['user_created']
df['user_duration_days'] = df['user_duration_days'].dt.days
df['user_duration_days'] = df['user_duration_days'].fillna(0).astype('int')

# Renamed 'date' column to 'tweet_date'
df.rename(columns={'date': 'tweet_date'}, inplace=True)
df.rename(columns={'text': 'tweet'}, inplace=True)

# Deleted columns user_created
df.drop(['user_created'], axis=1, inplace=True)

In [13]:
# Get rid of simple retweets
df = df.drop_duplicates(subset=['tweet'], keep='first')

### 2. Profiling

The dataset contains 60,504 unique tweets (simple retweets have been removed) referencing the hashtag #ChatGPT from 35,748 different users over the period December 5, 2002, through January 2, 2023.

In [14]:
# Unique tweets
df['tweet'].nunique()

60504

In [15]:
# Unique user_names
df['user_name'].nunique()

35748

In [16]:
# Earliest tweet in df
df['tweet_date'].min()

Timestamp('2022-12-05 17:08:20+0000', tz='UTC')

In [17]:
# Latest tweet in df
df['tweet_date'].max()

Timestamp('2023-01-02 20:50:45+0000', tz='UTC')

### 3. EDA

The vast majority of tweets occured in the early part of December, with the peak at December 6, when almost 7,000 tweets about ChatGPT were sent.  Since then, the number of tweets each day has declined significantly.  

In [29]:
# Create a df of tweets over time
tmp_df = df[['tweet', 'tweet_date']].copy()
tmp_df['Date'] = tmp_df['tweet_date'].dt.strftime("%Y-%m-%d")
pivot_table = tmp_df.pivot_table(
    index=['Date'],
    values=['tweet'],
    aggfunc={'tweet': ['count']}
)

In [30]:
# Create line chart
import plotly.express as px
fig = px.line(df_pivot, x='Date', y='tweet count', line_shape='spline')
fig.update_layout(
    title='Tweets Over Time', 
    xaxis = dict(
        showgrid=True, 
        rangeslider = dict(
            visible=True, 
            thickness=0.05
        )
    ), 
    yaxis = dict(
        showgrid=True
    ), 
    legend = dict(
        orientation='v'
    ), 
    paper_bgcolor='#FFFFFF'
)
fig.show(renderer="iframe")

ValueError: Value of 'y' is not the name of a column in 'data_frame'. Expected one of [('Date', ''), ('tweet', 'count')] but received: tweet count

Tweets mostly came from newer users to Twitter, but the distribution of user duration was bi-modal.  The vast majority of tweets were from brand new twitter users, but a significant proportion of tweets also came from users who've been on Twitter for 5000 days or more.

In [ ]:
fig = px.histogram(df, x='user_duration_days', histfunc='count')
fig.update_layout(
    title='Distribution of User Time on Twitter', 
    xaxis = dict(
        showgrid=True, 
        rangeslider = dict(
            visible=True, 
            thickness=0.05
        )
    ), 
    yaxis = dict(
        showgrid=True
    ), 
    legend = dict(
        orientation='v'
    ), 
    barmode='group', 
    paper_bgcolor='#FFFFFF'
)
fig.show(renderer="iframe")

Of the more than 60,000 tweets sent, only around 1,500 were from verified Twitter users.  Verified twitter users sending these tweets had more followers (147K vs. 3K), more friends (7K vs. 1K), and have been on Twitter longer (4600 vs. 2800 days) than non-Verified twitter users who sent the VAST majority of tweets about ChatGPT.  

In [ ]:
 # Pivoted into df
tmp_df = df[['user_verified', 'tweet']].copy()
pivot_table = tmp_df.pivot_table(
    index=['user_verified'],
    values=['tweet'],
    aggfunc={'tweet': ['count']}
)
pivot_table = pivot_table.set_axis([flatten_column_header(col) for col in pivot_table.keys()], axis=1)
df_pivot = pivot_table.reset_index()

# Deleted 1 row in df_pivot
df_pivot.drop(labels=[0], inplace=True)

# Pivoted into df
tmp_df = df[['user_followers', 'user_verified']].copy()
pivot_table = tmp_df.pivot_table(
    index=['user_verified'],
    values=['user_followers'],
    aggfunc={'user_followers': ['mean']}
)
pivot_table = pivot_table.set_axis([flatten_column_header(col) for col in pivot_table.keys()], axis=1)
df_pivot_1 = pivot_table.reset_index()

# Deleted 1 row in df_pivot_1
df_pivot_1.drop(labels=[0], inplace=True)

# Pivoted into df
tmp_df = df[['user_friends', 'user_verified']].copy()
pivot_table = tmp_df.pivot_table(
    index=['user_verified'],
    values=['user_friends'],
    aggfunc={'user_friends': ['mean']}
)
pivot_table = pivot_table.set_axis([flatten_column_header(col) for col in pivot_table.keys()], axis=1)
df_pivot_2 = pivot_table.reset_index()

# Deleted 1 row in df_pivot_2
df_pivot_2.drop(labels=[0], inplace=True)

# Pivoted into df
tmp_df = df[['user_verified', 'user_duration_days']].copy()
pivot_table = tmp_df.pivot_table(
    index=['user_verified'],
    values=['user_duration_days'],
    aggfunc={'user_duration_days': ['mean']}
)
pivot_table = pivot_table.set_axis([flatten_column_header(col) for col in pivot_table.keys()], axis=1)
df_pivot_3 = pivot_table.reset_index()

# Deleted 1 row in df_pivot_3
df_pivot_3.drop(labels=[0], inplace=True)

# Merged df_pivot and df_pivot_1 into df6
temp_df = df_pivot_1.drop_duplicates(subset=['user_verified']) # Remove duplicates so lookup merge only returns first match
df6 = df_pivot.merge(temp_df, left_on=['user_verified'], right_on=['user_verified'], how='left', suffixes=['_df_pivot', '_df_pivot_1'])

# Merged df6 and df_pivot_2 into df7
temp_df = df_pivot_2.drop_duplicates(subset=['user_verified']) # Remove duplicates so lookup merge only returns first match
df7 = df6.merge(temp_df, left_on=['user_verified'], right_on=['user_verified'], how='left', suffixes=['_df6', '_df_pivot_2'])

# Merged df7 and df_pivot_3 into df8
temp_df = df_pivot_3.drop_duplicates(subset=['user_verified']) # Remove duplicates so lookup merge only returns first match
df8 = df7.merge(temp_df, left_on=['user_verified'], right_on=['user_verified'], how='left', suffixes=['_df7', '_df_pivot_3'])

# Renamed df8 to Summary_Stats
Summary_Stats = df8
Summary_Stats
